In [15]:
# Assignment - 5
# Team name - Turing Machines
# Team members:
# Nikhil V Revankar - 01FB16ECS230
# Vikram G - 01FB16ECS484
# Vinayaka R Kamath - 01FB16ECS445

In [16]:
import pandas as pd
import numpy as np
import math
import warnings
warnings.filterwarnings('ignore')
from scipy.spatial.distance import cosine
data=pd.read_csv("train.csv")
data1=pd.read_csv("test.csv")

In [17]:
'''Question-1'''

'Question-1'

In [18]:
#a) average rating given by each user.
avg__rat_eachuser =pd.DataFrame({"mean_rating":data.groupby('user_id')['rating'].mean()}).reset_index()
print(avg__rat_eachuser)

      user_id  mean_rating
0         243     7.000000
1         244     7.000000
2         254     7.000000
3         805     8.000000
4        1075     8.000000
5        1131     7.333333
6        1261     6.000000
7        1424     7.333333
8        1435     9.000000
9        1733     5.000000
10       2197     9.000000
11       2295     6.000000
12       2766     8.000000
13       2977     7.000000
14       3346     9.000000
15       3363    10.000000
16       3373     9.000000
17       3467     9.000000
18       4017     9.000000
19       4571     7.500000
20       4785     9.000000
21       4795     9.000000
22       4938     8.500000
23       5048    10.000000
24       5091     8.000000
25       5439     8.500000
26       5476     4.000000
27       5539     9.500000
28       5582     9.500000
29       5741     8.000000
...       ...          ...
2487   274808     9.000000
2488   275020     9.000000
2489   275210     8.500000
2490   275306    10.000000
2491   275401     5.000000
2

In [19]:
#b) average rating given for each item.
avg_rat_eachitem =pd.DataFrame({"mean_rating":data.groupby('book')['rating'].mean()}).reset_index()
print(avg_rat_eachitem)

                                                 book  mean_rating
0                                 1st to Die: A Novel     7.607843
1           A Heartbreaking Work of Staggering Genius     7.488889
2   A Is for Alibi (Kinsey Millhone Mysteries (Pap...     7.560000
3   A Lesson Before Dying (Vintage Contemporaries ...     7.411765
4   A Man Named Dave: A Story of Triumph and Forgi...     7.885714
5   A Walk in the Woods: Rediscovering America on ...     8.531250
6                                   A Wrinkle In Time     8.837838
7                                      Absolute Power     7.800000
8            Ahab's Wife: Or, The Star-Gazer: A Novel     7.656250
9                                       American Gods     7.857143
10                    Angela's Ashes (MMP) : A Memoir     8.102041
11                                Bee Season: A Novel     7.863636
12                                 Bel Canto: A Novel     8.164179
13               Beloved (Plume Contemporary Fiction)     7.94

In [21]:
#c) Root Mean Squared Error. 
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())
tar=[]
average=data.rating.mean()
for i in range(len(data['rating'])):
        tar.append(average)
print("RMSE: ",rmse(np.array(data['rating']),np.array(tar)))

RMSE:  1.781336407021028


In [22]:
#d) 
def convert_to_matrix(data):
    #train data into matrix
    rows = data.user_id.unique()
    cols = data['book'].unique()
    data = data[['user_id', 'book', 'rating']]
    idict = dict(zip(cols, range(len(cols))))
    udict = dict(zip(rows, range(len(rows))))
    data.user_id = [ udict[i] for i in data.user_id ]
    data['book'] = [ idict[i] for i in data['book'] ]
    nmat= data.values
    return nmat,rows,cols,udict
    
#train-data into matrix
nmat_train, rows_train, cols_train,udict_train = convert_to_matrix(data)
print("Train-data Matrix:\n",nmat_train)
print("\nSparsity of train-data matrix :",(len([j for i in nmat_train for j in i if j==0])/(np.prod(nmat_train.shape))))
print("\n=============\n")

#test-data into matrix
nmat_test, rows_test, cols_test,udict_test = convert_to_matrix(data1)
print("Test-data Matrix:\n",nmat_test)
print("\nSparsity of test-data matrix :",len([j for i in nmat_test for j in i if j==0])/(np.prod(nmat_test.shape)))


Train-data Matrix:
 [[   0    0    5]
 [   1    1    9]
 [   2    2    6]
 ...
 [ 635   73   10]
 [ 283   41    8]
 [2516   78    7]]

Sparsity of train-data matrix : 0.015778357559945336


Test-data Matrix:
 [[  0   0   7]
 [  1   1   8]
 [  2   2   8]
 ...
 [846  51  10]
 [847  28   9]
 [419  46  10]]

Sparsity of test-data matrix : 0.006546644844517185


In [25]:
#e)
def fun(mat,rows,cols):   
    naive = np.zeros((len(rows),len(cols)))
    for row in mat:
        naive[row[0], row[1]] = row[2]
    amean1 = np.mean(naive[naive!=0])
    umean1 = sum(naive.T) / sum((naive!=0).T)
    imean1 = sum(naive) / sum((naive!=0))
    umean1 = umean1[~np.isnan(umean1)]
    imean1 = imean1[~np.isnan(imean1)]
    return naive,amean1,umean1,imean1
#for train_data
naive_train,amean1_train,umean1_train,imean1_train = fun(nmat_train,rows_train,cols_train)
#for test_data
naive_test,amean1_test,umean1_test,imean1_test = fun(nmat_test,rows_test,cols_test)

In [28]:
def predict_naive(imean,umean,amean,user, item):
    prediction = imean[item] + umean[user] - amean
    return prediction
predictions_train=[]
targets_train=[]
for row in nmat_train:
    user, item, actual = row[0], row[1], row[2]
    predictions_train.append(predict_naive(imean1_train,umean1_train,amean1_train,user, item))
    targets_train.append(actual)
print("(train_data)Root Mean Squared Error:",rmse(np.array(predictions_train),np.array(targets_train)))

(train_data)Root Mean Squared Error: 1.1444196071250665


In [29]:
'''Question-2'''

'Question-2'

In [30]:
#a
def cos(mat, a, b):
    if a == b:
        return 1
    aval = mat.T[a].nonzero()
    bval = mat.T[b].nonzero()
    corated = np.intersect1d(aval, bval)
    if len(corated) == 0:
        return 0
    avec = np.take(mat.T[a], corated)
    bvec = np.take(mat.T[b], corated)
    val = 1.0 - cosine(avec, bvec)
    if np.isnan(val):
        return 0
    return val

def pr(mat, a, b, imean):
    if a == b:
        return 1
    aval = mat.T[a].nonzero()
    bval = mat.T[b].nonzero()
    corated = np.intersect1d(aval, bval)
    if len(corated) < 2:
        return 0
    avec = np.take(mat.T[a], corated)
    bvec = np.take(mat.T[b], corated)
    avec1 = avec - imean[a]
    bvec1 = bvec - imean[b]
    val = 1.0 - cosine(avec1, bvec1)
    if np.isnan(val):
        return 0
    return val

def itemsimilar(mat, option):
    # *Calculate amean, umean and imean as before
    amean = np.mean(mat[mat!=0])
    umean = sum(mat.T) / sum((mat!=0).T)
    imean = sum(mat) / sum((mat!=0))
    n = mat.shape[1]
    # *initialize a zero matrix with dimensions n,n to get the similarity matrix
    sim_mat = np.zeros((n, n))
    if option == 'pr':
        #print("PR")
        for i in range(n):
            for j in range(n):
                sim_mat[i][j] = pr(mat, i, j, imean)
    elif option == 'cos':
        #print("COS")
        for i in range(n):
            for j in range(n):
                sim_mat[i][j] = cos(mat, i, j)
    else:
        #print("Default")
        sim_mat = cosine_similarity(mat.T)
    return sim_mat, amean, umean, imean

In [31]:
test_Isim_mat_cos, test_Iamean_cos, test_Iumean_cos, test_Iimean_cos = itemsimilar(naive_test,'cos')
test_Isim_mat_pr, test_Iamean_pr, test_Iumean_pr, test_Iimean_pr = itemsimilar(naive_test,'pr')

In [32]:
def usersimilar(mat,umean, option):
    n = mat.shape[1]
    # *initialize a zero matrix with dimensions n,n to get the similarity matrix
    sim_mat = np.zeros((n, n))
    if option == 'pr':
        #print("PR")
        for i in range(n):
            for j in range(n):
                sim_mat[i][j] = pr(mat, i, j, umean)
    elif option == 'cos':
        #print("COS")
        for i in range(n):
            for j in range(n):
                sim_mat[i][j] = cos(mat, i, j)
    else:
        #print("Default")
        sim_mat = cosine_similarity(mat.T)
    return sim_mat

In [33]:
test_Usim_mat_cos = usersimilar(naive_test.T,umean1_test,'cos')
test_Usim_mat_pr = usersimilar(naive_test.T,umean1_test,'pr')

In [34]:
K = 5
test_Ksimilar_cos = []
for x, i in enumerate(test_Usim_mat_cos):
    mi = i.argsort()
    mi = np.delete(mi, np.where(mi==x))
    test_Ksimilar_cos.append(mi[-K:])

test_Ksimilar_pr = []
for x,i in enumerate(test_Usim_mat_pr):
    mi = i.argsort()
    mi = np.delete(mi, np.where(mi==x))
    test_Ksimilar_pr.append(mi[-K:])
 

In [37]:
# example taking user 32 and finding k=5 similar users
user = 32
similar_users_cos=test_Ksimilar_cos[user]
similar_users_pr=test_Ksimilar_pr[user]
Ksimilar_userids_cos=[k for i in similar_users_cos for k,v in udict_test.items() if v==i]
Ksimilar_userids_pr=[k for i in similar_users_pr for k,v in udict_test.items() if v==i]
print("for user", user, ":", K, "similar user-ids using (cos) :", Ksimilar_userids_cos)
print("for user", user, ":", K, "similar user-ids using (pr) :", Ksimilar_userids_pr)


for user 32 : 5 similar user-ids using (cos) : [108405, 43246, 133706, 271558, 35859]
for user 32 : 5 similar user-ids using (pr) : [197659, 75591, 168144, 136139, 272573]


In [38]:
#b
def predict(user, item, mat, item_similarity, amean, umean, imean, k=20):
    nzero = mat[user].nonzero()[0]
    if len(nzero) == 0:
        return amean
    baseline = imean + umean[user] - amean
    choice = nzero[item_similarity[item, nzero].argsort()[::-1][:k]]
    prediction = ((mat[user, choice] - baseline[choice]).dot(item_similarity[item, choice])/ sum(item_similarity[item, choice])) + baseline[item]
    if np.isnan(prediction):
        prediction = amean
    if prediction > 10:
        prediction = 10
    if prediction < 1:
        prediction = 1
    return prediction

In [39]:
# example taking user 32 and item 1 and predicting the rate given by user 32
k= 20
user= 32
item = 1
print("Predicted rating for user -",user,"and item -",item,": ",predict(user, item, naive_test, test_Usim_mat_cos,amean1_test,umean1_test,imean1_test,k))

Predicted rating for user - 32 and item - 1 :  7.8207855973813425


In [40]:
#c
def get_results(train_data, test_data, option, rows, cols, k):
    #for train data
    full_mat = np.zeros((rows[0], cols[0]))
    for row in train_data:
        full_mat[row[0], row[1]] = row[2]
    item_similarity, amean, umean, imean = itemsimilar(full_mat, option)
    k = 10
    preds = np.zeros(len(train_data))
    if option == 'cos':
        for x, i in enumerate(train_data):
            preds[x] = predict(i[0], i[1], full_mat, item_similarity, amean, umean, imean, k)
    elif option == 'pr':
        for x, i in enumerate(train_data):
            preds[x] = predict(i[0], i[1], full_mat, item_similarity, amean, umean, imean, k)
    print('Train Error')
    print('RMSE : %.4f' % rmse(train_data[:, 2], preds))
    
    #for test data
    full_mat = np.zeros((rows[1], cols[1]))
    for row in test_data:
        full_mat[row[0], row[1]] = row[2]
    item_similarity, amean, umean, imean = itemsimilar(full_mat, option)
    k = 10
    preds = np.zeros(len(test_data))
    if option == 'cos':
        for x, i in enumerate(test_data):
            preds[x] = predict(i[0], i[1], full_mat, item_similarity, amean, umean, imean, k)
    elif option == 'pr':
        for x, i in enumerate(test_data):
            preds[x] = predict(i[0], i[1], full_mat, item_similarity, amean, umean, imean, k)
    print('Test Error')
    print('RMSE : %.4f' % rmse(test_data[:, 2], preds))

In [41]:
print("Train and Test Error using cos")
get_results(nmat_train, nmat_test, 'cos', (len(rows_train),len(rows_test)), (len(cols_train), len(cols_test)), k)
print("\n===========\n")
print("Train and Test Error using pr")
get_results(nmat_train, nmat_test, 'pr', (len(rows_train),len(rows_test)), (len(cols_train), len(cols_test)), k)

Train and Test Error using cos
Train Error
RMSE : 1.0102
Test Error
RMSE : 0.7641


Train and Test Error using pr
Train Error
RMSE : 1.3294
Test Error
RMSE : 0.8731


In [42]:
#d How do you find the optimal value of k while using kNN?
# In KNN, finding the value of k is not easy. 
# A small value of k means that noise will have a higher influence on the result and a large value make it computationally expensive.
# k must be  odd number if the number of classes is 2in order to avoid ties
# another simple approach to select k is set k=sqrt(n).